### With LDA extract topics from H.Clintons mails

In [ ]:
import pandas as pd
data_mail = pd.read_csv('/content/drive/My Drive/Emails.csv', header=0)
data_mail.head(5)

,Id,DocNumber,MetadataSubject,MetadataTo,MetadataFrom,SenderPersonId,MetadataDateSent,MetadataDateReleased,MetadataPdfLink,MetadataCaseNumber,MetadataDocumentClass,ExtractedSubject,ExtractedTo,ExtractedFrom,ExtractedCc,ExtractedDateSent,ExtractedCaseNumber,ExtractedDocNumber,ExtractedDateReleased,ExtractedReleaseInPartOrFull,ExtractedBodyText,RawText
0,1,C05739545,WOW,H,"Sullivan, Jacob J",87.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739545...,F-2015-04841,HRC_Email_296,FW: Wow,NaN,"Sullivan, Jacob J <Sullivan11@state.gov>",NaN,"Wednesday, September 12, 2012 10:16 AM",F-2015-04841,C05739545,05/13/2015,RELEASE IN FULL,NaN,UNCLASSIFIED\nU.S. Department of State\nCase N...
1,2,C05739546,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,H,NaN,NaN,2011-03-03T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739546...,F-2015-04841,HRC_Email_296,NaN,NaN,NaN,NaN,NaN,F-2015-04841,C05739546,05/13/2015,RELEASE IN PART,"B6\nThursday, March 3, 2011 9:45 PM\nH: Latest...",UNCLASSIFIED\nU.S. Department of State\nCase N...
2,3,C05739547,CHRIS STEVENS,;H,"Mills, Cheryl D",32.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739547...,F-2015-04841,HRC_Email_296,Re: Chris Stevens,B6,"Mills, Cheryl D <MillsCD@state.gov>","Abedin, Huma","Wednesday, September 12, 2012 11:52 AM",F-2015-04841,C05739547,05/14/2015,RELEASE IN PART,Thx,UNCLASSIFIED\nU.S. Department of State\nCase N...
3,4,C05739550,CAIRO CONDEMNATION - FINAL,H,"Mills, Cheryl D",32.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739550...,F-2015-04841,HRC_Email_296,FVV: Cairo Condemnation - Final,NaN,"Mills, Cheryl D <MillsCD@state.gov>","Mitchell, Andrew B","Wednesday, September 12,2012 12:44 PM",F-2015-04841,C05739550,05/13/2015,RELEASE IN PART,NaN,UNCLASSIFIED\nU.S. Department of State\nCase N...
4,5,C05739554,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,"Abedin, Huma",H,80.0,2011-03-11T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739554...,F-2015-04841,HRC_Email_296,NaN,NaN,NaN,NaN,NaN,F-2015-04841,C05739554,05/13/2015,RELEASE IN PART,"H <hrod17@clintonemail.com>\nFriday, March 11,...",B6\nUNCLASSIFIED\nU.S. Department of State\nCa...


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
lemmatizer = WordNetLemmatizer() 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
names = data_mail[pd.notnull(data_mail['MetadataFrom'])]
answer = []
for i in names['MetadataFrom']:
    ans = ''
    for char in i:
        if char.isalpha() or char == ' ':
            ans += char
        else:
            ans += ' '
    words = nltk.word_tokenize(ans)
    for w in words:
        answer.append(w.lower())
answer = set(answer)

In [ ]:
data = data_mail[pd.notnull(data_mail['ExtractedBodyText'])]

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
stop_words = set(stopwords.words('english'))
stop_words.update(['gov', 'clintonemail', 'september', 'february', 'march', 'october',
                   'january', 'december', 'june', 'saturday', 'tuesday', 'friday',
                   'monday', 'wednesday', 'thursday', 'august', 'april'])
stop_words.update(answer)
def preprocess2(rawtext):
    ans = ''
    for char in rawtext:
        if char.isalpha() or char == ' ':
            ans += char
        else:
            ans += ' '
    words = nltk.word_tokenize(ans)
    answer = []
    for w in words:
        w = lemmatizer.lemmatize(w.lower())
        if w not in stop_words and len(w)>2:
            answer.append(w.lower())
    
    return ' '.join(answer)

In [ ]:
def preprocess3(rawtext):
    ans = ''
    for char in rawtext:
        if char.isalpha() or char == ' ':
            ans += char
        else:
            ans += ' '
    words = nltk.word_tokenize(ans)
    answer = []
    for w in words:
        w = lemmatizer.lemmatize(w.lower())
        if w not in stop_words and len(w)>2:
            answer.append(w.lower())
    
    return answer

In [ ]:
from gensim import corpora,models, matutils
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models.ldamodel import LdaModel
vectorizer = TfidfVectorizer()
data["Clean"] = data["ExtractedBodyText"].apply(lambda x: preprocess2(x))
data["Clean_list"] = data["ExtractedBodyText"].apply(lambda x: preprocess3(x))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
print(data.sample(5)["Clean"])

5805    rage day rage noxious transformation conservat...
3693    fyi ended sending letter afternoon staff diffi...
4680                                        agreed worked
3421    yes sorry late response driving thanksgiving t...
4478                                           think fine
Name: Clean, dtype: object


In [ ]:
dictionary_tfidf = corpora.Dictionary(data["Clean_list"])
dictionary_tfidf.filter_extremes(no_below=5, no_above=0.5)

In [ ]:
bow_dict = [dictionary_tfidf.doc2bow(doc) for doc in data["Clean_list"]]
tfidf = models.TfidfModel(bow_dict)
corpus_tfidf = tfidf[bow_dict]

In [ ]:
from gensim import matutils
from gensim.models.ldamodel import LdaModel
model_lda = LdaModel(corpus_tfidf, passes=10,iterations = 100, num_topics=10, alpha='auto', id2word=dictionary_tfidf)
topics_to_str = [word for number, word in model_lda.print_topics()]
topics_to_str_split = list(map(lambda x: x.split("+"), topics_to_str))
topics_to_str_split  = [list(map(lambda x: x.split("*")[1].strip()[1:-1], elem)) for elem in topics_to_str_split]
for topic in topics_to_str_split :
    print(topic)

['calling', 'landed', 'happening', 'libya', 'benghazi', 'supposed', 'printing', 'isabelle', 'mini', 'track']
['jpg', 'scroll', 'updated', 'berry', 'glad', 'leaving', 'goy', 'wow', 'updating', 'mubarak']
['discus', 'said', 'tonight', 'say', 'message', 'add', 'statement', 'issue', 'first', 'woman']
['release', 'part', 'declassify', 'nice', 'fax', 'roger', 'reason', 'wireless', 'rob', 'blackberry']
['call', 'see', 'thx', 'yes', 'talk', 'get', 'tomorrow', 'pls', 'know', 'today']
['office', 'room', 'high', 'importance', 'department', 'arrive', 'route', 'depart', 'treaty', 'private']
['print', 'pls', 'assistant', 'please', 'dan', 'special', 'confidential', 'add', 'emailed', 'direct']
['sound', 'traffic', 'blackberry', 'reaction', 'bottom', 'via', 'share', 'mail', 'non', 'heading']
['fyi', 'iii', 'forwarded', 'remind', 'attached', 'correct', 'reminder', 'passed', 'raj', 'fuchsmh']
['checking', 'seen', 'sometime', 'offer', 'mark', 'mexico', 'bring', 'tonite', 'chair', 'front']


## Interpretation of  tf-idf
* ['call', 'see', 'thx', 'yes', 'talk', 'get', 'tomorrow', 'pls', 'know', 
'today'] - probably informal letters to someone close 
* ['discus', 'said', 'tonight', 'say', 'message', 'add', 'statement', 'issue', 'first', 'woman'] - discussion of statemens Hillary gave concerning women
* ['office', 'room', 'high', 'importance', 'department', 'arrive', 'route', 'depart', 'treaty', 'private'] - discussion of travels
* ['calling', 'landed', 'happening', 'libya', 'benghazi', 'supposed', 'printing', 'isabelle', 'mini', 'track'] - Judging by Libya and Benghazi, talking about the 2012 attacks on US diplomatic missions, at that time Hillary was Secretary of State
* ['print', 'pls', 'assistant', 'please', 'dan', 'special', 'confidential', 'add', 'emailed', 'direct'] maybe letters to assistants with tasks
* ['checking', 'seen', 'sometime', 'offer', 'mark', 'mexico', 'bring', 'tonite', 'chair', 'front'] - foreign policy issues?? in the visualization you can see more words for each cluster, and then there are Korea and Iraq


In [ ]:
!pip install pyLDAvis

In [ ]:
import pyLDAvis
import pyLDAvis.gensim

data_lda = pyLDAvis.gensim.prepare(model_lda, corpus_tfidf, dictionary_tfidf)

In [ ]:
pyLDAvis.enable_notebook()
pyLDAvis.display(data_lda)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=10, max_df=0.5)
data_vectorized = vectorizer.fit_transform(data["Clean"])

In [ ]:
import gensim
from scipy.sparse import csc, csr_matrix
corpus_cv = matutils.Sparse2Corpus(data_vectorized.T,documents_columns=True)
vecdict=dict([(i, s) for i, s in enumerate(vectorizer.get_feature_names())])

In [ ]:
dictionary_cv = gensim.corpora.Dictionary.from_corpus(corpus_cv, vecdict)

In [ ]:
model_lda = LdaModel(corpus_cv, passes=20,iterations = 100, num_topics=10, alpha='auto', id2word=dictionary_cv)
topics_to_str = [word for number, word in model_lda.print_topics()]
topics_to_str_split = list(map(lambda x: x.split("+"), topics_to_str))
topics_to_str_split  = [list(map(lambda x: x.split("*")[1].strip()[1:-1], elem)) for elem in topics_to_str_split]
for topic in topics_to_str_split :
    print(topic)

['fyi', 'print', 'doc', 'date', 'subject', 'benghazi', 'information', 'sensitive', 'dept', 'produced']
['party', 'republican', 'vote', 'election', 'would', 'percent', 'democrat', 'said', 'conservative', 'voter']
['tomorrow', 'thx', 'yes', 'may', 'sunday', 'talk', 'today', 'morning', 'sent', 'time']
['government', 'security', 'united', 'would', 'people', 'effort', 'country', 'iran', 'force', 'support']
['obama', 'president', 'israel', 'israeli', 'said', 'policy', 'administration', 'palestinian', 'mcchrystal', 'time']
['one', 'say', 'year', 'day', 'people', 'time', 'like', 'said', 'woman', 'right']
['call', 'add', 'holbrooke', 'letter', 'pls', 'ops', 'list', 'called', 'want', 'send']
['office', 'meeting', 'room', 'department', 'arrive', 'route', 'depart', 'conference', 'private', 'residence']
['china', 'policy', 'health', 'development', 'economic', 'world', 'company', 'image', 'food', 'woman']
['see', 'know', 'get', 'think', 'good', 'would', 'also', 'want', 'work', 'need']


## Interpratation of lda с count vectorize
* ['call', 'add', 'holbrooke', 'letter', 'pls', 'ops', 'list', 'called', 'want', 'send'] informal mails
* ['tomorrow', 'thx', 'yes', 'may', 'sunday', 'talk', 'today', 'morning', 'sent', 'time'] close to previous. but slightly more formal
* ['party', 'republican', 'vote', 'election', 'would', 'percent', 'democrat', 'said', 'conservative', 'voter'] - elections
* ['obama', 'president', 'israel', 'israeli', 'said', 'policy', 'administration', 'palestinian', 'mcchrystal', 'time'] - palestine-israel conflict
* ['government', 'security', 'united', 'would', 'people', 'effort', 'country', 'iran', 'force', 'support'] - relations between Iran and the United States, it can be seen in the visualization that the words nuclear and settlement are both included here
* ['one', 'say', 'year', 'day', 'people', 'time', 'like', 'said', 'woman', 'right'] - women rights
* ['office', 'meeting', 'room', 'department', 'arrive', 'route', 'depart', 'conference', 'private', 'residence'] - some kind of logistics, meetings and departures
* ['china', 'policy', 'health', 'development', 'economic', 'world', 'company', 'image', 'food', 'woman'] - relationship with China? Hillary also defended women's rights in China, but perhaps this is a strtch

In [ ]:
data_lda = pyLDAvis.gensim.prepare(model_lda, corpus_cv, dictionary_cv)
pyLDAvis.enable_notebook()
pyLDAvis.display(data_lda)